In [1]:
dbutils.library.installPyPI('snowflake-connector-python', version = "2.0.2")
dbutils.library.installPyPI("azure-storage-blob", version = "2.1.0")
dbutils.library.installPyPI("dill", version="0.2.9")
dbutils.library.installPyPI("category_encoders", version="2.2.2")
dbutils.library.installPyPI("lightgbm", version="2.3.0")
dbutils.library.installPyPI("scipy", version="1.2.1")
dbutils.library.installPyPI("mlflow", version="1.9.1")
dbutils.library.installPyPI("pandas", version="0.21.0")
dbutils.library.installPyPI('numpy', version="1.14.0")
dbutils.library.installPyPI("scikit-learn", version="0.18.2")


dbutils.library.restartPython()

PyPI package snowflake-connector-python has been installed already. The previous installed package is PyPI:(snowflake-connector-python)-(2.0.2)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package azure-storage-blob has been installed already. The previous installed package is PyPI:(azure-storage-blob)-(2.1.0)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package dill has been installed already. The previous installed package is PyPI:(dill)-(0.2.9)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
PyPI package category_encoders has been installed already. The previous installed package is PyPI:(category_encoders)-(2.2.2)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.

In [2]:
import pandas as pd
import os
from os import path
from scipy.stats import pearsonr, spearmanr
from scipy import stats
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import log_loss, accuracy_score, average_precision_score, confusion_matrix, f1_score, roc_auc_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# import snowflake.connector
import dill
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
import mlflow
from sklearn.metrics import make_scorer
# import pingouin as pg

In [3]:
import sklearn
import category_encoders
import lightgbm
import scipy
print('pd', pd.__version__)
print('sklearn', sklearn.__version__)
print('dill', dill.__version__)
print('category_encoders', category_encoders.__version__)
print('lightgbm', lightgbm.__version__)
print('numpy', np.__version__)
print('scipy', scipy.__version__)

pd 0.21.0
sklearn 0.18.2
dill 0.2.9
category_encoders 2.2.2
lightgbm 2.3.0
numpy 1.14.0
scipy 1.2.1

In [4]:
if mlflow.get_tracking_uri() != 'databricks':
  print('update tracking uri')
  mlflow.set_tracking_uri('databricks')

In [5]:
def _get_key_metrics(y, y_pred, y_pred_proba):
  accuracy = accuracy_score(y, y_pred)
  roc_auc = roc_auc_score(y, y_pred_proba)
  pr_auc = average_precision_score(y, y_pred)
  logloss = log_loss(y, y_pred_proba)
  mae = mean_absolute_error(y, y_pred)
  conf_max = confusion_matrix(y, y_pred)/len(y)

  return accuracy, roc_auc, pr_auc, logloss, mae
  
  
def eval_model(y_train, y_train_pred, y_train_pred_proba, y_test, y_test_pred, y_test_pred_proba): 
  y_train = np.reshape(y_train.astype('float').values, (len(y_train), ))
  y_test = np.reshape(y_test.astype('float').values, (len(y_test), ))
  
  print('in y_train, pct of ones', y_train.mean(), 'pct of zeros', 1-y_train.mean())  
  print('in y_test, pct of ones', y_test.mean(), 'pct of zeros', 1-y_test.mean())
  
  residual_std_train = (y_train_pred_proba - y_train).std()
  y_train_std = y_train.std()
  
  residual_std_test = (y_test_pred_proba - y_test).std()
  y_test_std = y_test.std()
  
  accuracy_train, roc_auc_train, pr_auc_train, log_loss_train, mae_train = _get_key_metrics(y_train, y_train_pred, y_train_pred_proba)
  accuracy_test, roc_auc_test, pr_auc_test, log_loss_test, mae_test = _get_key_metrics(y_test, y_test_pred, y_test_pred_proba)
  
  std_shrinking_test = (residual_std_test - y_test_std)/y_test_std
  std_shrinking_train = (residual_std_train - y_train_std)/y_train_std
  
#     logloss_test_pred = log_loss(y_test_reshaped, y_pred_proba)
#   logloss_baseline = log_loss(y_test_reshaped, np.ones(len(y_pred)))
#   pred_random = np.random.randint(2, size=len(y_test_reshaped))
#   logloss_baseline = log_loss(y_test_reshaped, pred_random)
#   print('logloss_baseline', logloss_baseline)
#   print('logloss_test_pred', logloss_test_pred)
  
#   acc = accuracy_score(y_test_reshaped, y_pred)
#   f1 = f1_score(y_test_reshaped, y_pred)  
#   roc_auc = roc_auc_score(y_test_reshaped, y_pred_proba)
#   print('acc', acc)
#   print('average_precision_score predicted', average_precision_score(y_test_reshaped, y_pred_proba), 'average_precision_score all zeros', average_precision_score(y_test_reshaped, np.zeros(len(y_pred))))
#   print('f1 score', f1)
#   print('roc_auc_score', roc_auc)
  
    
  return std_shrinking_test, std_shrinking_train, \
         residual_std_test, y_test_std, residual_std_train, y_train_std, \
         accuracy_test, roc_auc_test, pr_auc_test, log_loss_test, mae_test, \
         accuracy_train, roc_auc_train, pr_auc_train, log_loss_train, mae_train

#### global variables

In [7]:
number_of_chunks = 10

## Data loading

#### Real-time query snowflake using complicated sql

In [10]:
sql_query_train = """
with daypart_mapping as (
  SELECT 
    case when day_part = 'late_night' then 'latenight' 
      else DAY_PART
    END as daypart,
    MIN(LOCAL_HOUR) as min_hour,
    MAX(LOCAL_HOUR) as max_hour
  FROM PRODDB.STATIC.LOOKUP_DAY_PART_MAPPING 
  GROUP BY 1
),
flf_targets as (
SELECT 
  FLF.STARTING_POINT_ID, 
  FLF.STARTING_POINT_NAME, 
  FLF.TIME_OF_DAY as daypart, 
  dp.min_hour,
  dp.max_hour,
  FLF.TARGET_IDEAL_FLF, 
  FLF.MIN_TARGET_FLF_RANGE, 
  FLF.MAX_TARGET_FLF_RANGE, 
  FLF.TARGET_CREATED_AT, 
  LEAD(FLF.TARGET_CREATED_AT, 1) OVER (PARTITION BY FLF.STARTING_POINT_ID, FLF.TIME_OF_DAY ORDER BY FLF.TARGET_CREATED_AT) as next, 
  IFNULL(next, '2022-01-01') AS NEXT_TARGET_CREATED_DATE
FROM STATIC.LOOKUP_TARGET_FLF_BY_REGION flf
LEFT JOIN daypart_mapping dp 
  on flf.TIME_OF_DAY = dp.DAYPART
),
flf_raw as (
SELECT
  dd.created_at,
  dd.DELIVERY_ID,
  dd.active_date,
  dd.STORE_STARTING_POINT_ID,
  dd.SUBMARKET_ID,
  dd.flf,
  fces.num_delivered as num_delivered,
  fces.num_opened as num_opened,
  sm.LAUNCH_DATE as submarket_launch_date,
  convert_timezone('UTC', dd.TIMEZONE, dd.CREATED_AT) as created_at_local,
  TO_DATE(created_at_local) as created_at_local_date,
  hour(created_at_local) * 2 + floor(minute(created_at_local)/30.0) as window_id,
  datediff('second', dd.CREATED_AT, dd.ACTUAL_DELIVERY_TIME)/60.0 as asap,
  dd.DISTINCT_ACTIVE_DURATION/60.0 as dat,
  datediff('second', dd.DASHER_CONFIRMED_TIME, dd.DASHER_AT_STORE_TIME)/60.0 as d2r,
  case when datediff('second', dd.QUOTED_DELIVERY_TIME, dd.ACTUAL_DELIVERY_TIME)/60 > 20 then 1 else 0 END as lateness_20_min,
  flf.daypart,
  case when dd.flf - flf.MAX_TARGET_FLF_RANGE > 0 then 1 else 0 END as is_flf_above_max,
  case when dd.flf - flf.TARGET_IDEAL_FLF > 0 then 1 else 0 END as is_flf_above_ideal
FROM PRODDB.PUBLIC.DIMENSION_DELIVERIES dd 
LEFT JOIN flf_targets flf
  on dd.STORE_STARTING_POINT_ID = flf.STARTING_POINT_ID
  AND hour(convert_timezone('UTC', dd.TIMEZONE, dd.created_at)) between flf.min_hour and flf.max_hour
  AND convert_timezone('UTC', dd.TIMEZONE, dd.created_at) between flf.TARGET_CREATED_AT and flf.NEXT_TARGET_CREATED_DATE
LEFT JOIN PRODDB.PUBLIC.MAINDB_SUBMARKET sm 
  ON dd.SUBMARKET_ID = sm.ID
LEFT JOIN public.fact_cx_email_summary fces
  ON dd.SUBMARKET_ID = fces.SUBMARKET_ID
  AND convert_timezone('UTC',dd.timezone,
 dateadd('minute',cast(floor(date_part('minute',dd.created_at) / 30) * 30 as int), date_trunc('hour',dd.created_at))
 ) = fces.half_hour_local
WHERE dd.created_at between '2020-03-16' and '2020-04-27'
  AND dd.IS_FILTERED_CORE = true 
  AND dd.IS_ASAP = true 
  AND dd.IS_CONSUMER_PICKUP = false 
  AND fulfillment_type != 'merchant_fleet'
),

flf_raw_grouped as(
SELECT
    t.created_at_local_date,
    t.DAYPART,
    t.STORE_STARTING_POINT_ID,
    AVG(t.ASAP) as avg_asap,
    AVG(t.DAT) as avg_dat,
    AVG(t.D2R) as avg_d2r,
    AVG(t.IS_FLF_ABOVE_IDEAL) as avg_is_flf_above_ideal,
    AVG(t.flf) as avg_flf,
    AVG(t.num_opened) as avg_num_opened,
    AVG(t.num_delivered) as avg_num_delivered
FROM flf_raw t
GROUP BY t.created_at_local_date, t.DAYPART, t.STORE_STARTING_POINT_ID
),
flf_hist as(
SELECT 
    t1.created_at_local_date,
    dayofweek(t1.CREATED_AT_LOCAL) as DAY_OF_WEEK,
    hour(t1.CREATED_AT_LOCAL) as HOUR_OF_DAY,
    t1.STORE_STARTING_POINT_ID,
    t1.SUBMARKET_ID,  
    t1.DAYPART,
    t1.WINDOW_ID,
    t1.is_flf_above_ideal
FROM flf_raw t1
LEFT JOIN flf_raw_grouped t2
    ON t1.created_at_local_date = DATEADD(Day, -15, t2.created_at_local_date)
    AND t1.DAYPART = t2.DAYPART
    AND t1.STORE_STARTING_POINT_ID = t2.STORE_STARTING_POINT_ID
)

SELECT *
FROM flf_hist
SAMPLE(20)
"""

# sql_query_test_verify = """
# CREATE TEMP TABLE CHIZHANG.DAYPART_MAPPING AS (
#   SELECT 
#     CASE WHEN day_part = 'late_night' THEN 'latenight' 
#       ELSE DAY_PART
#     END AS daypart,
#     MIN(LOCAL_HOUR) AS min_hour,
#     MAX(LOCAL_HOUR) AS max_hour
#   FROM PRODDB.STATIC.LOOKUP_DAY_PART_MAPPING 
#   GROUP BY 1
# );

# CREATE temp TABLE CHIZHANG.FLF_TARGETS AS (
# SELECT 
#   FLF.STARTING_POINT_ID, 
#   FLF.STARTING_POINT_NAME, 
#   FLF.TIME_OF_DAY AS daypart, 
#   dp.min_hour,
#   dp.max_hour,
#   FLF.TARGET_IDEAL_FLF, 
#   FLF.MIN_TARGET_FLF_RANGE, 
#   FLF.MAX_TARGET_FLF_RANGE, 
#   FLF.TARGET_CREATED_AT, 
#   LEAD(FLF.TARGET_CREATED_AT, 1) OVER (PARTITION BY FLF.STARTING_POINT_ID, FLF.TIME_OF_DAY ORDER BY FLF.TARGET_CREATED_AT) AS NEXT, 
#   IFNULL(NEXT, '2022-01-01') AS NEXT_TARGET_CREATED_DATE
# FROM STATIC.LOOKUP_TARGET_FLF_BY_REGION flf
# LEFT JOIN CHIZHANG.DAYPART_MAPPING dp 
#   ON flf.TIME_OF_DAY = dp.DAYPART
# );
    
# CREATE temp TABLE CHIZHANG.FLF_RAW AS (
# SELECT
#   dd.created_at,
#   dd.DELIVERY_ID,
#   dd.active_date,
#   dd.STORE_STARTING_POINT_ID,
#   dd.SUBMARKET_ID,
#   dd.flf,
#   fces.num_delivered AS num_delivered,
#   fces.num_opened AS num_opened,
#   sm.LAUNCH_DATE AS submarket_launch_date,
#   convert_timezone('UTC', dd.TIMEZONE, dd.CREATED_AT) AS created_at_local,
#   TO_DATE(created_at_local) AS created_at_local_date,
#   hour(created_at_local) * 2 + floor(minute(created_at_local)/30.0) AS window_id,
#   datediff('second', dd.CREATED_AT, dd.ACTUAL_DELIVERY_TIME)/60.0 AS asap,
#   dd.DISTINCT_ACTIVE_DURATION/60.0 AS dat,
#   datediff('second', dd.DASHER_CONFIRMED_TIME, dd.DASHER_AT_STORE_TIME)/60.0 as d2r,
#   CASE WHEN datediff('second', dd.QUOTED_DELIVERY_TIME, dd.ACTUAL_DELIVERY_TIME)/60 > 20 THEN 1 ELSE 0 END AS lateness_20_min,
#   flf.daypart,
#   CASE WHEN dd.flf - flf.MAX_TARGET_FLF_RANGE > 0 THEN 1 ELSE 0 END AS is_flf_above_max,
#   CASE WHEN dd.flf - flf.TARGET_IDEAL_FLF > 0 THEN 1 ELSE 0 END AS is_flf_above_ideal
# FROM PRODDB.PUBLIC.DIMENSION_DELIVERIES dd 
# LEFT JOIN CHIZHANG.FLF_TARGETS flf
#   ON dd.STORE_STARTING_POINT_ID = flf.STARTING_POINT_ID
#   AND hour(convert_timezone('UTC', dd.TIMEZONE, dd.created_at)) BETWEEN flf.min_hour AND flf.max_hour
#   AND convert_timezone('UTC', dd.TIMEZONE, dd.created_at) BETWEEN flf.TARGET_CREATED_AT AND flf.NEXT_TARGET_CREATED_DATE
# LEFT JOIN PRODDB.PUBLIC.MAINDB_SUBMARKET sm 
#   ON dd.SUBMARKET_ID = sm.ID
# LEFT JOIN public.fact_cx_email_summary fces
#   ON dd.SUBMARKET_ID = fces.SUBMARKET_ID
#   AND convert_timezone('UTC',dd.timezone,
#  dateadd('minute',CAST(floor(date_part('minute',dd.created_at) / 30) * 30 AS INT), date_trunc('hour',dd.created_at))
#  ) = fces.half_hour_local
# //  WHERE CAST(DD.CREATED_AT as DATE) >= dateadd('DAY', -3, TO_TIMESTAMP_NTZ(LOCALTIMESTAMP)) 
#   WHERE dd.created_at BETWEEN '2020-03-16' AND '2020-04-27'
#   AND CAST(DD.CREATED_AT as DATE) < dateadd('DAY', 0, TO_TIMESTAMP_NTZ(LOCALTIMESTAMP))  
#   AND dd.IS_FILTERED_CORE = true 
#   AND dd.IS_ASAP = true 
#   AND dd.IS_CONSUMER_PICKUP = false 
#   AND fulfillment_type != 'merchant_fleet'
# );

# CREATE TEMP TABLE CHIZHANG.FLF_RAW_GROUPED AS(
# SELECT
#     t.created_at_local_date,
#     t.DAYPART,
#     t.STORE_STARTING_POINT_ID,
#     AVG(t.ASAP) AS avg_asap,
#     AVG(t.DAT) AS avg_dat,
#     AVG(t.D2R) AS avg_d2r,
#     AVG(t.IS_FLF_ABOVE_IDEAL) AS avg_is_flf_above_ideal,
#     AVG(t.flf) AS avg_flf,
#     AVG(t.num_opened) AS avg_num_opened,
#     AVG(t.num_delivered) AS avg_num_delivered
# FROM CHIZHANG.FLF_RAW t
# GROUP BY t.created_at_local_date, t.DAYPART, t.STORE_STARTING_POINT_ID
# );

# SELECT 
#     -- t1.CREATED_AT,
#     -- t1.DELIVERY_ID, 
#     -- t1.ACTIVE_DATE, 
#     dayofweek(t1.CREATED_AT_LOCAL) as DAY_OF_WEEK,
#     hour(t1.CREATED_AT_LOCAL) as HOUR_OF_DAY,
#     t1.STORE_STARTING_POINT_ID,
#     t1.SUBMARKET_ID,
#     -- t1.FLF,
#     -- t1.NUM_DELIVERED, 
#     -- t1.NUM_OPENED,
#     -- t1.SUBMARKET_LAUNCH_DATE, 
#     -- t1.CREATED_AT_LOCAL, 
#     -- t1.CREATED_AT_LOCAL_DATE,
#     -- t1.ASAP, 
#     -- t1.DAT,
#     -- t1.D2R, 
#     -- t1.LATENESS_20_MIN,
#     t1.DAYPART,
#     t1.WINDOW_ID
#     -- t1.IS_FLF_ABOVE_MAX, 
#     -- t1.IS_FLF_ABOVE_IDEAL,
#     -- t2.avg_asap,
#     -- t2.avg_dat,
#     -- t2.avg_d2r,
#     -- t2.avg_is_flf_above_ideal,
#     -- t2.avg_flf,
#     -- t2.avg_num_delivered,
#     -- t2.avg_num_opened
# FROM CHIZHANG.FLF_RAW t1
# LEFT JOIN CHIZHANG.FLF_RAW_GROUPED t2
#     ON t1.created_at_local_date = DATEADD(Day, -15, t2.created_at_local_date)
#     AND t1.DAYPART = t2.DAYPART
#     AND t1.STORE_STARTING_POINT_ID = t2.STORE_STARTING_POINT_ID
# SAMPLE(5)
# """


In [11]:
# full query. complicated query from snowflake
# toPandas is time-consuming
scope_name = 'chizhang-scope'
pw_key_name = 'snowflake-password'
un_key_name = 'snowflake-user'
user = dbutils.secrets.get(scope=scope_name, key=un_key_name)
password = dbutils.secrets.get(scope=scope_name, key=pw_key_name)

# snowflake connection options
options = dict(sfurl="doordash.snowflakecomputing.com/",
               sfaccount="DOORDASH",
               sfuser=user,
               sfpassword=password,
               sfdatabase="PRODDB",
               sfschema="public",
               sfwarehouse="ADHOC")

print(user)
print(password)

# sql_load_table = """ select * from variance_reduction_flf_train_test """
all_data = spark.read.format("snowflake").options(**options).option("query", sql_query_train).load()
raw_data = all_data.toPandas()

#### Query pre-generated table in snowflake using simple sql

In [13]:
# orig + weather + supply/demand
# sql_load_pre_gen_table = """select * from flf_weather_supply_demand sample (20)""" 
# orig + weather + supply/demand + pw_flf + incentive
sql_load_pre_gen_table = """select * from flf_weather_supply_demand_hist_incentive sample (20)"""

In [14]:
# simple query.
# load pre-generated table from snowflake

user = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-user")
password = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-password")

os.environ['SNOWFLAKE_USER'] = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-user")
os.environ['SNOWFLAKE_PW'] = dbutils.secrets.get(scope="chizhang-scope", key="snowflake-password")

# snowflake connection options
params = dict(
  user=os.environ['SNOWFLAKE_USER'],
  password=os.environ['SNOWFLAKE_PW'],
  account='DOORDASH',
  database='PRODDB',
  warehouse='ADHOC',
  schema='public',
)

#8min to query 20% of 65mm rows
with snowflake.connector.connect(**params) as ctx:
  raw_data = pd.read_sql(sql_load_pre_gen_table, ctx)

#### save csv to dbfs

In [16]:
raw_data_mem_size = raw_data.info(memory_usage='deep')
print('raw_data_mem_size', raw_data_mem_size)

#15min to save
[df_i.to_csv('/dbfs/chizhang/variance_reduction/data/data_weather_supply_demand_flf_incentive_pw_{id}.csv'.format(id=id)) for id, df_i in  enumerate(np.array_split(raw_data, number_of_chunks))]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13160232 entries, 0 to 13160231
Data columns (total 30 columns):
CREATED_AT datetime64[ns]
DELIVERY_ID int64
DAY_OF_WEEK int64
HOUR_OF_DAY int64
DAYPART object
STORE_STARTING_POINT_ID int64
SUBMARKET_ID int64
WINDOW_ID int64
IS_FLF_ABOVE_IDEAL int64
PW_HOURLY_AVG_FLF float64
PW_HOURLY_AVG_IS_FLF_ABOVE_IDEAL float64
PW_HOURLY_AVG_IS_FLF_ABOVE_MAX float64
PW_HOURLY_AVG_DASHER_PAY_OTHER float64
HH_TEMPERATURE float64
HH_APPARENT_TEMPERATURE float64
HH_PRESSURE float64
HH_HUMIDITY float64
HH_VISIBILITY float64
HH_WIND_SPEED float64
HH_CLOUD_COVER float64
HH_DEWPOINT float64
HH_HOURLY_WEATHER_SUMMARY object
HH_PRECIP_INTENSITY float64
HH_PRECIP_PROBABILITY float64
HH_ICON object
HH_PRECIP_ACCUMULATION float64
HH_PRECIP_TYPE object
PRED_DEMAND float64
ACTUAL_DEMAND float64
UNDER_PREDICTED_DEMAND float64
dtypes: datetime64[ns](1), float64(18), int64(7), object(4)
memory usage: 5.5 GB
raw_data_mem_size None
Out[7]: [None, None, None, None, None, None, None, None, None, None]

#### load from dbfs

In [18]:
#2min to load 10 files
dir = r'/dbfs/chizhang/variance_reduction/data' # use your path

li = []

for id in range(number_of_chunks):
  print('chunk id', id)
  filename = dir + '/data_weather_supply_demand_flf_incentive_pw_'+str(id)+'.csv'
  df = pd.read_csv(filename, index_col=None)
  li.append(df)

raw_data = pd.concat(li, axis=0, ignore_index=True)
print(raw_data.shape)

chunk id 0
chunk id 1
chunk id 2
chunk id 3
chunk id 4
chunk id 5
/databricks/python/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2741: DtypeWarning: Columns (22,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
 interactivity=interactivity, compiler=compiler)
chunk id 6
chunk id 7
chunk id 8
chunk id 9
(13160232, 31)

In [19]:
df = raw_data.copy()
df = df.reindex()
print('data size', df.shape)
df.columns = map(str.lower, df.columns)
df.head(5)

,unnamed: 0,created_at,delivery_id,day_of_week,hour_of_day,daypart,store_starting_point_id,submarket_id,window_id,is_flf_above_ideal,...,hh_dewpoint,hh_hourly_weather_summary,hh_precip_intensity,hh_precip_probability,hh_icon,hh_precip_accumulation,hh_precip_type,pred_demand,actual_demand,under_predicted_demand
0,0,2020-04-28 17:45:45.873103,564542301,2,13,lunch,704,78,27,0,...,40.97,Partly Cloudy,0.0,0.0,partly-cloudy-day,NaN,NaN,59.679853,71.0,11.320147
1,1,2020-04-28 17:39:13.605148,564526926,2,13,lunch,704,78,27,0,...,40.97,Partly Cloudy,0.0,0.0,partly-cloudy-day,NaN,NaN,59.679853,71.0,11.320147
2,2,2020-04-28 17:49:24.791669,564550788,2,13,lunch,704,78,27,0,...,40.97,Partly Cloudy,0.0,0.0,partly-cloudy-day,NaN,NaN,59.679853,71.0,11.320147
3,3,2020-04-28 17:59:39.664271,564575019,2,13,lunch,704,78,27,0,...,40.97,Partly Cloudy,0.0,0.0,partly-cloudy-day,NaN,NaN,59.679853,71.0,11.320147
4,4,2020-04-28 17:39:31.263843,564527641,2,13,lunch,704,78,27,0,...,40.97,Partly Cloudy,0.0,0.0,partly-cloudy-day,NaN,NaN,59.679853,71.0,11.320147


In [20]:
# drop cols with >50% na
print(df.shape)
df.columns
df_nan = df.isna().sum()/len(df)
print(df_nan)
cols_to_remove = list(df_nan[df_nan > 0.4].index)
print('we drop cols with >50% missing values:', cols_to_remove)
df = df.drop(cols_to_remove, axis=1)
print(df.shape)


for c in ['pw_hourly_avg_flf', 'pw_hourly_avg_is_flf_above_ideal', 'pw_hourly_avg_is_flf_above_max', 'pw_hourly_avg_dasher_pay_other']:
  df = df[df[c].notna()]

print(df.isna().sum()/len(df))
print(df.shape)

(13160232, 31)
unnamed: 0 0.000000
created_at 0.000000
delivery_id 0.000000
day_of_week 0.000000
hour_of_day 0.000000
daypart 0.000004
store_starting_point_id 0.000000
submarket_id 0.000000
window_id 0.000000
is_flf_above_ideal 0.000000
pw_hourly_avg_flf 0.115376
pw_hourly_avg_is_flf_above_ideal 0.115373
pw_hourly_avg_is_flf_above_max 0.115373
pw_hourly_avg_dasher_pay_other 0.115373
hh_temperature 0.053842
hh_apparent_temperature 0.053842
hh_pressure 0.053842
hh_humidity 0.053842
hh_visibility 0.053842
hh_wind_speed 0.053842
hh_cloud_cover 0.053842
hh_dewpoint 0.053842
hh_hourly_weather_summary 0.053842
hh_precip_intensity 0.053842
hh_precip_probability 0.053842
hh_icon 0.053842
hh_precip_accumulation 0.998414
hh_precip_type 0.461473
pred_demand 0.000000
actual_demand 0.000000
under_predicted_demand 0.000000
dtype: float64
we drop cols with >50% missing values: ['hh_precip_accumulation', 'hh_precip_type']
(13160232, 29)
unnamed: 0 0.000000
created_at 0.000000
delivery_id 0.000000
day_of_week 0.000000
hour_of_day 0.000000
daypart 0.000000
store_starting_point_id 0.000000
submarket_id 0.000000
window_id 0.000000
is_flf_above_ideal 0.000000
pw_hourly_avg_flf 0.000000
pw_hourly_avg_is_flf_above_ideal 0.000000
pw_hourly_avg_is_flf_above_max 0.000000
pw_hourly_avg_dasher_pay_other 0.000000
hh_temperature 0.060767
hh_apparent_temperature 0.060767
hh_pressure 0.060767
hh_humidity 0.060767
hh_visibility 0.060767
hh_wind_speed 0.060767
hh_cloud_cover 0.060767
hh_dewpoint 0.060767
hh_hourly_weather_summary 0.060767
hh_precip_intensity 0.060767
hh_precip_probability 0.060767
hh_icon 0.060767
pred_demand 0.000000
actual_demand 0.000000
under_predicted_demand 0.000000
dtype: float64
(11641856, 29)

In [21]:
df.columns

Out[ 11 ]: Index(['unnamed: 0', 'created_at', 'delivery_id', 'day_of_week', 'hour_of_day',
 'daypart', 'store_starting_point_id', 'submarket_id', 'window_id',
 'is_flf_above_ideal', 'pw_hourly_avg_flf',
 'pw_hourly_avg_is_flf_above_ideal', 'pw_hourly_avg_is_flf_above_max',
 'pw_hourly_avg_dasher_pay_other', 'hh_temperature',
 'hh_apparent_temperature', 'hh_pressure', 'hh_humidity',
 'hh_visibility', 'hh_wind_speed', 'hh_cloud_cover', 'hh_dewpoint',
 'hh_hourly_weather_summary', 'hh_precip_intensity',
 'hh_precip_probability', 'hh_icon', 'pred_demand', 'actual_demand',
 'under_predicted_demand'],
 dtype='object')

## Feature creating

In [23]:
# combine real-time features with historical aggregated features
# features_cat = ['day_of_week', 'hour_of_day', 'STORE_STARTING_POINT_ID', 'SUBMARKET_ID', 'LATENESS_20_MIN', 'DAYPART', '30min'] #30 min, sp, submarket, ->market level feats
# historical aggregated features := avg(feat_values) over SP_ID/(DATE_LOCAL-15days)/DAYPART units (i.e., 14 days ago) 
# features_num = ['AVG_ASAP', 'AVG_DAT', 'AVG_D2R', 'AVG_FLF', 'AVG_IS_FLF_ABOVE_IDEAL']

# features_cat = ['day_of_week', 'hour_of_day', 'store_starting_point_id', 'submarket_id', 'daypart', 'window_id'] #30 min, sp, submarket, ->market level feats
# # features_num = ['AVG_ASAP', 'AVG_DAT', 'AVG_D2R', 'AVG_FLF', 'AVG_IS_FLF_ABOVE_IDEAL', 'AVG_NUM_DELIVERED', 'AVG_NUM_OPENED']
# features_num = []

# features_cat = ['day_of_week', 'hour_of_day', 'store_starting_point_id', 'submarket_id', 'daypart', 'window_id', \
#                'hh_hourly_weather_summary',  'hh_precip_type', 'hh_icon']
# features_num = ['hh_temperature', 'hh_apparent_temperature', 'hh_pressure', 'hh_humidity', 'hh_dewpoint', 'hh_visibility', \
#                 'hh_wind_speed', 'hh_cloud_cover', 'hh_dewpoint', 'hh_precip_intensity', 'hh_precip_probability', \
#                 'pred_demand']


features_orig = ['day_of_week', 'hour_of_day', 'daypart', 'store_starting_point_id', 'submarket_id', 'window_id']
features_weather = ['hh_temperature', 'hh_apparent_temperature', 'hh_pressure', 'hh_humidity', 'hh_visibility', \
                    'hh_wind_speed', 'hh_cloud_cover', 'hh_dewpoint', 'hh_precip_intensity', 'hh_precip_probability',\
                    'hh_hourly_weather_summary', 'hh_icon']
features_supply_demand = ['pred_demand', 'actual_demand', 'under_predicted_demand']
features_hist_flf = ['pw_hourly_avg_flf', 'pw_hourly_avg_is_flf_above_ideal', 'pw_hourly_avg_is_flf_above_max']
features_incentive = ['pw_hourly_avg_dasher_pay_other']

features_type = { "day_of_week" : "category",
                  "hour_of_day" : "category",
                  'daypart'     : "category",
                 
                  'hh_hourly_weather_summary': "category", #or target_encode
                  'hh_icon'     : "category",#or target_encode

                  'store_starting_point_id' : "target_encode", 
                  'submarket_id' : "target_encode",   
                  'window_id'    : "target_encode",

                  'hh_temperature' : "numerical", 
                  'hh_apparent_temperature' : "numerical", 
                  'hh_pressure'    : "numerical",
                  'hh_humidity'    : "numerical",
                  'hh_visibility'  : "numerical",
                  'hh_wind_speed'  : "numerical",
                  'hh_cloud_cover' : "numerical", 
                  'hh_dewpoint'    : "numerical",
                  'hh_precip_intensity'   : "numerical",
                  'hh_precip_probability' : "numerical",
                 
                  'pw_hourly_avg_flf' : "numerical", 
                  'pw_hourly_avg_is_flf_above_ideal': "numerical",
                  'pw_hourly_avg_is_flf_above_max': "numerical",
                 
                  'pw_hourly_avg_dasher_pay_other' : "numerical",
                 }

for f in features_type:
  if features_type.get(f) == 'category' or features_type.get(f) == 'target_encode':
    df[f] = df[f].astype('category')
  elif features_type.get(f) == 'numerical':
    df[f] = df[f].astype('float')
  else:
    raise('wrong feature type')
    
features_target_encode = [k for k,v in features_type.items() if v == 'target_encode']

features = features_orig + features_weather + features_supply_demand + features_hist_flf + features_incentive
metrics = 'is_flf_above_ideal'

In [24]:
len(features)

Out[ 22 ]: 25

## Eval raw correlation

In [26]:
df.columns

Out[ 13 ]: Index(['unnamed: 0', 'created_at', 'delivery_id', 'day_of_week', 'hour_of_day',
 'daypart', 'store_starting_point_id', 'submarket_id', 'window_id',
 'is_flf_above_ideal', 'pw_hourly_avg_flf',
 'pw_hourly_avg_is_flf_above_ideal', 'pw_hourly_avg_is_flf_above_max',
 'pw_hourly_avg_dasher_pay_other', 'hh_temperature',
 'hh_apparent_temperature', 'hh_pressure', 'hh_humidity',
 'hh_visibility', 'hh_wind_speed', 'hh_cloud_cover', 'hh_dewpoint',
 'hh_hourly_weather_summary', 'hh_precip_intensity',
 'hh_precip_probability', 'hh_icon', 'pred_demand', 'actual_demand',
 'under_predicted_demand'],
 dtype='object')

## Plot raw correlation

In [28]:
def plot_corr(df):
  te = TargetEncoder(cols=features_target_encode)
  df_corr = te.fit_transform(df[features], df[metrics])
#   print(df_corr)
  df_corr['is_flf_above_ideal'] = df[metrics]
  print(df_corr.corr()['is_flf_above_ideal'].abs().sort_values(ascending=False))

plot_corr(df)


is_flf_above_ideal 1.000000
pw_hourly_avg_is_flf_above_ideal 0.329422
store_starting_point_id 0.316617
pw_hourly_avg_is_flf_above_max 0.283884
submarket_id 0.231601
window_id 0.204695
pw_hourly_avg_flf 0.194348
under_predicted_demand 0.150332
pw_hourly_avg_dasher_pay_other 0.103802
hh_visibility 0.049783
hh_precip_probability 0.038691
hh_wind_speed 0.036426
hh_pressure 0.033314
hh_apparent_temperature 0.031012
actual_demand 0.026766
hh_temperature 0.025377
hh_humidity 0.025292
hh_precip_intensity 0.014922
hh_cloud_cover 0.012332
hh_dewpoint 0.010239
pred_demand 0.002703
Name: is_flf_above_ideal, dtype: float64

## Train/test data preparing

In [30]:
from datetime import datetime, timedelta

df['created_at'] = pd.to_datetime(df['created_at'])
date_train_end = df['created_at'].dt.date.max() - timedelta(weeks=1)
date_train_start = date_train_end - timedelta(weeks=5)

train_index = df[(df['created_at'] < str(date_train_end)) & (df['created_at'] >= str(date_train_start))].index
test_index = df[df['created_at'] >= str(date_train_end)].index

print('training period:', df.loc[train_index]['created_at'].min(), df.loc[train_index]['created_at'].max())
print('testing period:', df.loc[test_index]['created_at'].min(), df.loc[test_index]['created_at'].max())


training period: 2020-04-27 00:00:00.025144 2020-05-21 23:59:59.818855
testing period: 2020-05-22 00:00:00.137764 2020-05-29 23:59:59.805666

In [31]:
# check train and test index do not intersects
assert len(train_index.intersection(test_index))==0, 'data leakage'
# check if training and testing data has 7 days a week
assert df.loc[test_index]['day_of_week'].nunique() == 7, 'testing data does not have 7 days a week'
assert df.loc[train_index]['day_of_week'].nunique() == 7, 'training data does not have 7 days a week'

### Create pipeline & train

### random search mlflow

In [34]:
mlflow.set_experiment('/Shared/Experiments/variance_reduction_flf')

def train_mlflow_randomsearch(features, run_name):
  with mlflow.start_run(run_name=run_name):

      X_train = df.loc[train_index, features]
      y_train = df.loc[train_index, metrics]

      X_test = df.loc[test_index, features]
      y_test = df.loc[test_index, metrics]

      print("x_train shape", X_train.shape, 'y_train shape', y_train.shape)
      print("x_test shape", X_test.shape, 'y_test shape', y_test.shape)

      
      # target encoding
      teppl = TargetEncoder(cols=features_target_encode)
      # clf
      clfppl = LGBMClassifier()

      # random search
      # best for orig+weather+supply_demand
      # {'clf__boosting_type': 'gbdt', 'clf__learning_rate': 0.10799590819985533, 'clf__num_leaves': 476}
      # {'clf__boosting_type': 'gbdt', 'clf__learning_rate': 0.10442678580364907, 'clf__num_leaves': 411} +weather
      # {'clf__boosting_type': 'gbdt', 'clf__learning_rate': 0.08908617527882136, 'clf__num_leaves': 491} +weather+supply/demand
      
      randomParams = {
          'clf__learning_rate': stats.uniform(0.01, 0.1), 
          'clf__num_leaves': stats.randint(16, 512),
          'clf__boosting_type' : ['gbdt', 'dart', 'goss']    
      }
  
      
      ppl = Pipeline([('target_encode', teppl), ('clf', clfppl)])

      random_search = RandomizedSearchCV(ppl, randomParams, n_jobs=-1, verbose=2, n_iter=50)
      random_search.fit(X_train, y_train)
      
      y_test_pred = random_search.predict(X_test)
      y_test_pred_proba = random_search.predict_proba(X_test)[:,1]
      
      y_train_pred = random_search.predict(X_train)
      y_train_pred_proba = random_search.predict_proba(X_train)[:,1]
      

      # metrics
      std_shrinking_test, std_shrinking_train, \
           residual_std_test, y_test_std, residual_std_train, y_train_std, \
           accuracy_test, roc_auc_test, pr_auc_test, log_loss_test, mae_test, \
           accuracy_train, roc_auc_train, pr_auc_train, log_loss_train, mae_train = eval_model(y_train, y_train_pred, y_train_pred_proba, y_test, y_test_pred, y_test_pred_proba)

      print('best param', random_search.best_params_)

      mlflow.set_tag('features', features)

      print("Test Log Loss: {0}".format(log_loss_test))
      print("Train Log Loss: {0}".format(log_loss_train))

      print("Test Accuracy Score: {0}".format(accuracy_test))
      print("Train Accuracy Score: {0}".format(accuracy_train))

      print("Test ROC AUC: {0}".format(roc_auc_test))
      print("Train ROC AUC: {0}".format(roc_auc_train))

      print("Test PR AUC: {0}".format(pr_auc_test))
      print("Train PR AUC: {0}".format(pr_auc_train))

      print("Test MAE: {0}".format(mae_test))
      print("Train MAE: {0}".format(mae_train))


      mlflow.log_metric('Test residual_std', residual_std_test)
      mlflow.log_metric('Test y std', y_test_std)
      mlflow.log_metric('Test shrinking', std_shrinking_test)

      mlflow.log_metric('Train residual std', residual_std_train)
      mlflow.log_metric('Train y std', y_train_std)
      mlflow.log_metric('Train shrinking', std_shrinking_train)

      mlflow.log_metric('accuracy_train', accuracy_train)
      mlflow.log_metric('accuracy_test', accuracy_test)

      mlflow.log_metric('train_pr_auc', pr_auc_train)
      mlflow.log_metric('test_pr_auc', pr_auc_test)

      mlflow.log_metric('train_roc_auc', roc_auc_train)
      mlflow.log_metric('test_roc_auc', roc_auc_test)
      
      mlflow.log_param("clf__boosting_type", clf__boosting_type)
      mlflow.log_param("clf__learning_rate", clf__learning_rate)
      mlflow.log_param("clf__num_leaves", clf__num_leaves)
      
  return random_search

In [35]:
train_mlflow_randomsearch(features_orig + features_weather + features_supply_demand + features_hist_flf + features_incentive, run_name='orig_weather_supplydemand_histflf_incentive_p2w_random_search')

--------------------------------------------------------------------------- 
 TerminatedWorkerError Traceback (most recent call last)
 <command-171461> in <module> () 
 ----> 1 train_mlflow_randomsearch ( features_orig + features_weather + features_supply_demand + features_hist_flf + features_incentive , run_name = 'orig_weather_supplydemand_histflf_incentive_p2w_random_search' ) 

 <command-171460> in train_mlflow_randomsearch (features, run_name) 
 35 
 36 random_search = RandomizedSearchCV ( ppl , randomParams , n_jobs = - 1 , verbose = 2 , n_iter = 50 ) 
 ---> 37 random_search . fit ( X_train , y_train ) 
 38 
 39 y_test_pred = random_search . predict ( X_test ) 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/sklearn/model_selection/_search.py in fit (self, X, y, groups, **fit_params) 
 708 return results 
 709 
 --> 710 self . _run_search ( evaluate_candidates ) 
 711 
 712 # For multi-metric evaluation, store the best_index_, best_params_ and 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/sklearn/model_selection/_search.py in _run_search (self, evaluate_candidates) 
 1482 evaluate_candidates(ParameterSampler(
 1483 self . param_distributions , self . n_iter , 
 -> 1484 random_state=self.random_state))
 
 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/sklearn/model_selection/_search.py in evaluate_candidates (candidate_params) 
 687 for parameters , ( train , test ) 
 688 in product(candidate_params,
 --> 689 cv.split(X, y, groups)))
 690 
 691 if len ( out ) < 1 : 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/joblib/parallel.py in __call__ (self, iterable) 
 1015 
 1016 with self . _backend . retrieval_context ( ) : 
 -> 1017 self . retrieve ( ) 
 1018 # Make sure that we get a last message telling us we are done 
 1019 elapsed_time = time . time ( ) - self . _start_time 

 /usr/lib/python3.5/contextlib.py in __exit__ (self, type, value, traceback) 
 75 value = type ( ) 
 76 try : 
 ---> 77 self . gen . throw ( type , value , traceback ) 
 78 raise RuntimeError ( "generator didn't stop after throw()" ) 
 79 except StopIteration as exc : 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/joblib/_parallel_backends.py in retrieval_context (self) 
 151 tasks . 
 152 """
 --> 153 yield 
 154 
 155 def _prepare_worker_env ( self , n_jobs ) : 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/joblib/parallel.py in __call__ (self, iterable) 
 1015 
 1016 with self . _backend . retrieval_context ( ) : 
 -> 1017 self . retrieve ( ) 
 1018 # Make sure that we get a last message telling us we are done 
 1019 elapsed_time = time . time ( ) - self . _start_time 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/joblib/parallel.py in retrieve (self) 
 907 try : 
 908 if getattr ( self . _backend , 'supports_timeout' , False ) : 
 --> 909 self . _output . extend ( job . get ( timeout = self . timeout ) ) 
 910 else : 
 911 self . _output . extend ( job . get ( ) ) 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-79c94572-1bd7-41f2-9252-5cff47c539b9/lib/python3.5/site-packages/joblib/_parallel_backends.py in wrap_future_result (future, timeout) 
 560 AsyncResults.get from multiprocessing."""
 561 try : 
 --> 562 return future . result ( timeout = timeout ) 
 563 except LokyTimeoutError : 
 564 raise TimeoutError ( ) 

 /usr/lib/python3.5/concurrent/futures/_base.py in result (self, timeout) 
 403 raise CancelledError ( ) 
 404 elif self . _state == FINISHED : 
 --> 405 return self . __get_result ( ) 
 406 else : 
 407 raise TimeoutError ( ) 

 /usr/lib/python3.5/concurrent/futures/_base.py in __get_r

In [36]:
df.loc[train_index, features].head(5)

,day_of_week,hour_of_day,daypart,store_starting_point_id,submarket_id,window_id,hh_temperature,hh_apparent_temperature,hh_pressure,hh_humidity,...,hh_precip_probability,hh_hourly_weather_summary,hh_icon,pred_demand,actual_demand,under_predicted_demand,pw_hourly_avg_flf,pw_hourly_avg_is_flf_above_ideal,pw_hourly_avg_is_flf_above_max,pw_hourly_avg_dasher_pay_other
0,2,13,lunch,704,78,27,61.81,61.81,1022.1,0.46,...,0.0,Partly Cloudy,partly-cloudy-day,59.679853,71.0,11.320147,1.764056,0.288136,0.0,35.169492
1,2,13,lunch,704,78,27,61.81,61.81,1022.1,0.46,...,0.0,Partly Cloudy,partly-cloudy-day,59.679853,71.0,11.320147,1.764056,0.288136,0.0,35.169492
2,2,13,lunch,704,78,27,61.81,61.81,1022.1,0.46,...,0.0,Partly Cloudy,partly-cloudy-day,59.679853,71.0,11.320147,1.764056,0.288136,0.0,35.169492
3,2,13,lunch,704,78,27,61.81,61.81,1022.1,0.46,...,0.0,Partly Cloudy,partly-cloudy-day,59.679853,71.0,11.320147,1.764056,0.288136,0.0,35.169492
4,2,13,lunch,704,78,27,61.81,61.81,1022.1,0.46,...,0.0,Partly Cloudy,partly-cloudy-day,59.679853,71.0,11.320147,1.764056,0.288136,0.0,35.169492


#### random search mlflow customized scoring

In [38]:
mlflow.set_experiment('/Shared/Experiments/variance_reduction_flf')

def train_mlflow_randomsearch_scoring(features):
  with mlflow.start_run(run_name='weather_randomsearch_1'):

      X_train = df.loc[train_index, features]
      y_train = df.loc[train_index, metrics]

      X_test = df.loc[test_index, features]
      y_test = df.loc[test_index, metrics]

      print("x_train shape", X_train.shape, 'y_train shape', y_train.shape)
      print("x_test shape", X_test.shape, 'y_test shape', y_test.shape)

      
      # target encoding
      teppl = TargetEncoder(cols=features_target_encode)
      # clf
      clfppl = LGBMClassifier()

      # random search
      # best for orig+weather+supply_demand
      # {'clf__boosting_type': 'gbdt', 'clf__learning_rate': 0.10799590819985533, 'clf__num_leaves': 476}
      # {'clf__boosting_type': 'gbdt', 'clf__learning_rate': 0.10442678580364907, 'clf__num_leaves': 411} +weather
      # {'clf__boosting_type': 'gbdt', 'clf__learning_rate': 0.08908617527882136, 'clf__num_leaves': 491} +weather+supply/demand
      
      randomParams = {
          'clf__learning_rate': stats.uniform(0.01, 0.1), 
          'clf__num_leaves': stats.randint(16, 512),
          'clf__boosting_type' : ['gbdt', 'dart', 'goss']    
      }
      
      #new scoring function
#     def custom_loss_func_est(estimator, X_train, y_train):
#         y_pred = estimator.predict_proba(X_train)[:, 1]
#         df_y_pred = pd.DataFrame(data=y_pred, columns=['pred_proba'])
#         df_tmp = pd.concat([X_train, y_train, df_y_pred], axis=1, sort=False)
#         res = pg.partial_corr(data=df_tmp, x='pred_proba', y=metrics, covar=['submarket_id', 'pw_hourly_avg_is_flf_above_ideal']).round(3)
#         return abs(res['r'].values[0])

#     custom_scorer_partial_corr = make_scorer(custom_loss_func_est, greater_is_better=True)
    
      def my_scorer(estimator, xx_train, yy_true):
        covar = ['pw_hourly_avg_is_flf_above_ideal', 'pw_hourly_avg_flf']
        yy_pred = estimator.predict_proba(xx_train)[:, 1]
        df_y_pred = pd.DataFrame(data=yy_pred, columns=['pred_proba'])
        df_yy_true = pd.DataFrame(data=yy_true.values, columns=['label'])
       
        df_tmp = pd.concat([xx_train[covar], df_yy_true, df_y_pred], axis=1, sort=False)
#         print(xx_train.head(5))
#         print(df_yy_true.head(5))
#         print(df_y_pred.head(5))                                                    
        print(df_tmp.head(100))
                                                    
        res = pg.partial_corr(data=df_tmp, x='pred_proba', y='label', covar=covar).round(1) #3
        return abs(res['r'].values[0])
  
      
      ppl = Pipeline([('target_encode', teppl), ('clf', clfppl)])

      random_search = RandomizedSearchCV(ppl, randomParams, n_jobs=-1, verbose=2, n_iter=50, scoring=my_scorer)
      random_search.fit(X_train, y_train)
      
      y_test_pred = random_search.predict(X_test)
      y_test_pred_proba = random_search.predict_proba(X_test)[:,1]

      y_train_pred = random_search.predict(X_train)
      y_train_pred_proba = random_search.predict_proba(X_train)[:,1]

      std_shrinking_test, std_shrinking_train, \
           residual_std_test, y_test_std, residual_std_train, y_train_std, \
           accuracy_test, roc_auc_test, pr_auc_test, log_loss_test, mae_test, \
           accuracy_train, roc_auc_train, pr_auc_train, log_loss_train, mae_train = eval_model(y_train, y_train_pred, y_train_pred_proba, y_test, y_test_pred, y_test_pred_proba)

      mlflow.set_tag('features', features)

      print("Test Log Loss: {0}".format(log_loss_test))
      print("Train Log Loss: {0}".format(log_loss_train))

      print("Test Accuracy Score: {0}".format(accuracy_test))
      print("Train Accuracy Score: {0}".format(accuracy_train))

      print("Test ROC AUC: {0}".format(roc_auc_test))
      print("Train ROC AUC: {0}".format(roc_auc_train))

      print("Test PR AUC: {0}".format(pr_auc_test))
      print("Train PR AUC: {0}".format(pr_auc_train))

      print("Test MAE: {0}".format(mae_test))
      print("Train MAE: {0}".format(mae_train))


      mlflow.log_metric('Test residual_std', residual_std_test)
      mlflow.log_metric('Test y std', y_test_std)
      mlflow.log_metric('Test shrinking', std_shrinking_test)

      mlflow.log_metric('Train residual std', residual_std_train)
      mlflow.log_metric('Train y std', y_train_std)
      mlflow.log_metric('Train shrinking', std_shrinking_train)


      mlflow.log_metric('accuracy_train', accuracy_train)
      mlflow.log_metric('accuracy_test', accuracy_test)

      mlflow.log_metric('train_pr_auc', pr_auc_train)
      mlflow.log_metric('test_pr_auc', pr_auc_test)

      mlflow.log_metric('train_roc_auc', roc_auc_train)
      mlflow.log_metric('test_roc_auc', roc_auc_test)
      
      mlflow.log_parameter("clf__boosting_type", clf__boosting_type)
      mlflow.log_parameter("clf__learning_rate", clf__learning_rate)
      mlflow.log_parameter("clf__num_leaves", clf__num_leaves)
      
      print(random_search.best_params_)
  return random_search

In [39]:
# train_mlflow_randomsearch(features_orig)
# train_mlflow_randomsearch(features_orig + features_weather)
# train_mlflow_randomsearch(features_orig + features_supply_demand)
random_search = train_mlflow_randomsearch(features_orig + features_weather + features_supply_demand + features_hist_flf + features_incentive)

--------------------------------------------------------------------------- 
 _RemoteTraceback Traceback (most recent call last)
 _RemoteTraceback : 
"""
Traceback (most recent call last):
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 431, in _process_worker
 r = call_item()
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 285, in __call__
 return self.fn(*self.args, **self.kwargs)
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 593, in __call__
 return self.func(*args, **kwargs)
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/joblib/parallel.py", line 253, in __call__
 for func, args, kwargs in self.items]
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/joblib/parallel.py", line 253, in <listcomp>
 for func, args, kwargs in self.items]
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 544, in _fit_and_score
 test_scores = _score(estimator, X_test, y_test, scorer)
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 591, in _score
 scores = scorer(estimator, X_test, y_test)
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 89, in __call__
 score = scorer(estimator, *args, **kwargs)
 File "<command-169831>", line 55, in my_scorer
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/pingouin/correlation.py", line 639, in partial_corr
 assert data.shape[0] > 2, 'Data must have at least 3 non-NAN samples.'
AssertionError: Data must have at least 3 non-NAN samples.
"""

The above exception was the direct cause of the following exception:

 AssertionError Traceback (most recent call last)
 <command-169832> in <module> 
 2 # train_mlflow_randomsearch(features_orig + features_weather) 
 3 # train_mlflow_randomsearch(features_orig + features_supply_demand) 
 ----> 4 random_search = train_mlflow_randomsearch ( features_orig + features_weather + features_supply_demand + features_hist_flf + features_incentive ) 

 <command-169831> in train_mlflow_randomsearch (features) 
 60 
 61 random_search = RandomizedSearchCV ( ppl , randomParams , n_jobs = - 1 , verbose = 2 , n_iter = 50 , scoring = my_scorer ) 
 ---> 62 random_search . fit ( X_train , y_train ) 
 63 
 64 y_test_pred = random_search . predict ( X_test ) 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/sklearn/model_selection/_search.py in fit (self, X, y, groups, **fit_params) 
 708 return results
 709 
 --> 710 self . _run_search ( evaluate_candidates ) 
 711 
 712 # For multi-metric evaluation, store the best_index_, best_params_ and 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/sklearn/model_selection/_search.py in _run_search (self, evaluate_candidates) 
 1482 evaluate_candidates(ParameterSampler(
 1483 self . param_distributions , self . n_iter , 
 -> 1484 random_state=self.random_state))
 
 /local_disk0/pythonVirtualEnvDirs/virtualEnv-d0f2420c-6167-48df-a0bc-eec45cf62acb/lib/python3.7/site-packages/sklearn/model_selection/_search.py in evaluate_candidates (candidate_params) 
 687 for parameters , ( train , test ) 
 688 in product(candidate_params,
 --> 689 cv.split(X, y, groups)))
 690 
 691 if le

In [40]:
X_test = df.loc[test_index, features]
y_test = df.loc[test_index, metrics]
y_test_pred_prob = random_search.predict_proba(X_test)[:, 1]

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-169833> in <module> () 
 1 X_test = df . loc [ test_index , features ] 
 2 y_test = df . loc [ test_index , metrics ] 
 ----> 3 y_test_pred_prob = random_search . predict_proba ( X_test ) [ : , 1 ] 

 NameError : name 'random_search' is not defined

In [41]:
y_test_pred_prob.shape, X_test.shape
test_result = df.loc[test_index].copy()
test_result['pred_prob'] = y_test_pred_prob
print(test_result.head(5))
test_result.to_csv('/dbfs/chizhang/variance_reduction/result/test_result_weather_supply_demand.csv')

In [42]:
test_result = pd.read_csv('/dbfs/chizhang/variance_reduction/result/test_result_weather_supply_demand.csv')

In [43]:
metrics = ['is_flf_above_ideal']
test_result['pred_prob'].std(), test_result['is_flf_above_ideal'].std()

Out[11]: (0.18630403969930892, 0.3783979991160973)

In [44]:
(test_result['pred_prob'] - test_result['is_flf_above_ideal']).std()

Out[10]: 0.3428744368825526

#### random search

In [46]:
# target encoding
teppl = TargetEncoder(cols=features_target_encode)
# clf
clfppl = LGBMClassifier()

# random search
randomParams = {
    'clf__learning_rate': stats.uniform(0.01, 0.1), 
    'clf__num_leaves': stats.randint(16, 512),
    'clf__boosting_type' : ['gbdt', 'dart', 'goss']    
}

ppl = Pipeline([('target_encode', teppl), ('clf', clfppl)])

random_search = RandomizedSearchCV(ppl, randomParams, n_jobs=-1, verbose=2, n_iter=100)
random_search.fit(X_train, y_train)

In [47]:
random_search.best_params_

In [48]:
ppl_best = random_search.best_estimator_

#### best model after randome search

In [50]:
teppl = TargetEncoder(cols=features_target_encode)

clfppl = LGBMClassifier()

ppl_best = Pipeline([('target_encode', teppl), ('clf', clfppl)])
ppl_best.set_params(clf__learning_rate=0.09985392034627226, clf__boosting_type='gbdt', clf__num_leaves=486)


X_train = df.loc[train_index, features]
y_train = df.loc[train_index, metrics]

X_test = df.loc[test_index, features]
y_test = df.loc[test_index, metrics]

      
ppl_best.fit(X_train, y_train)

Out[ 20 ]: Pipeline(steps=[('target_encode', TargetEncoder(cols=['store_starting_point_id', 'submarket_id', 'window_id'],
 drop_invariant=False, handle_missing='value',
 handle_unknown='value', min_samples_leaf=1, return_df=True,
 smoothing=1.0, verbose=0)), ('clf', LGBMClassifier(boosting_type='gbdt'...ambda=0.0,
 silent=True, subsample=1.0, subsample_for_bin=200000,
 subsample_freq=0))])

#### clf vanilla

In [52]:
mlflow.set_experiment('/Shared/Experiments/variance_reduction_flf')

def train_mlflow(features, run_name):
  with mlflow.start_run(run_name=run_name):

      X_train = df.loc[train_index, features]
      y_train = df.loc[train_index, metrics]

      X_test = df.loc[test_index, features]
      y_test = df.loc[test_index, metrics]

      print("x_train shape", X_train.shape, 'y_train shape', y_train.shape)
      print("x_test shape", X_test.shape, 'y_test shape', y_test.shape)

      teppl_vanilla = TargetEncoder(cols=features_target_encode)
      clfppl_vanilla = LGBMClassifier()

      ppl_vanilla = Pipeline([('target_encode_vanilla', teppl_vanilla), ('clf_vanilla', clfppl_vanilla)])

      ppl_vanilla.fit(X_train, y_train)

      y_test_pred = ppl_vanilla.predict(X_test)
      y_test_pred_proba = ppl_vanilla.predict_proba(X_test)[:,1]

      y_train_pred = ppl_vanilla.predict(X_train)
      y_train_pred_proba = ppl_vanilla.predict_proba(X_train)[:,1]

      
      # save to csv
      test_result = df.loc[test_index].copy()
      test_result['pred_prob'] = y_test_pred_proba
      print(test_result.head(5))
      test_result.to_csv('/dbfs/chizhang/variance_reduction/result/test_result_'+ run_name+ '_0610.csv')
      
      std_shrinking_test, std_shrinking_train, \
           residual_std_test, y_test_std, residual_std_train, y_train_std, \
           accuracy_test, roc_auc_test, pr_auc_test, log_loss_test, mae_test, \
           accuracy_train, roc_auc_train, pr_auc_train, log_loss_train, mae_train = eval_model(y_train, y_train_pred, y_train_pred_proba, y_test, y_test_pred, y_test_pred_proba)

      mlflow.set_tag('features', features)

      print("Test Log Loss: {0}".format(log_loss_test))
      print("Train Log Loss: {0}".format(log_loss_train))

      print("Test Accuracy Score: {0}".format(accuracy_test))
      print("Train Accuracy Score: {0}".format(accuracy_train))


      print("Test ROC AUC: {0}".format(roc_auc_test))
      print("Train ROC AUC: {0}".format(roc_auc_train))

      print("Test PR AUC: {0}".format(pr_auc_test))
      print("Train PR AUC: {0}".format(pr_auc_train))

      print("Test MAE: {0}".format(mae_test))
      print("Train MAE: {0}".format(mae_train))

      mlflow.log_metric('Test residual_std', residual_std_test)
      mlflow.log_metric('Test y std', y_test_std)
      mlflow.log_metric('Test shrinking', std_shrinking_test)

      mlflow.log_metric('Train residual std', residual_std_train)
      mlflow.log_metric('Train y std', y_train_std)
      mlflow.log_metric('Train shrinking', std_shrinking_train)


      mlflow.log_metric('accuracy_train', accuracy_train)
      mlflow.log_metric('accuracy_test', accuracy_test)

      mlflow.log_metric('train_pr_auc', pr_auc_train)
      mlflow.log_metric('test_pr_auc', pr_auc_test)

      mlflow.log_metric('train_roc_auc', roc_auc_train)
      mlflow.log_metric('test_roc_auc', roc_auc_test)
      
  return ppl_vanilla

In [53]:
# train_mlflow(features_orig)
# train_mlflow(features_orig + features_weather)
# train_mlflow(features_orig + features_supply_demand)
# train_mlflow(features_orig + features_weather + features_supply_demand)
ppl_vanilla = train_mlflow(features_orig + features_weather + features_supply_demand + features_hist_flf + features_incentive, run_name='orig_weather_supplydemand_histflf_incentive_p1w')

x_train shape (8745379, 25) y_train shape (8745379,)
x_test shape (2896477, 25) y_test shape (2896477,)
/local_disk0/pythonVirtualEnvDirs/virtualEnv-5eaa5bd1-eded-41c6-8aac-6a40c293e8bd/lib/python3.5/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
/local_disk0/pythonVirtualEnvDirs/virtualEnv-5eaa5bd1-eded-41c6-8aac-6a40c293e8bd/lib/python3.5/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 if diff:
 unnamed: 0 created_at delivery_id day_of_week \
1581 1581 2020-05-29 11:43:03.189071 638041813 5 
1582 1582 2020-05-29 11:58:43.382475 638045930 5 
1583 1583 2020-05-29 11:57:54.267532 638045700 5 
1584 1584 2020-05-29 11:00:08.028520 638033149 5 
1585 1585 2020-05-29 11:27:46.033317 638038349 5 

 hour_of_day daypart store_starting_point_id submarket_id window_id \
1581 7 breakfast 6925 23 15 
1582 7 breakfast 6925 23 15 
1583 21 latenight 5997 2279 43 
1584 21 latenight 5997 2279 42 
1585 21 latenight 5997 2279 42 

 is_flf_above_ideal ... hh_cloud_cover hh_dewpoint \
1581 1 ... 0.78 67.14 
1582 1 ... 0.78 67.14 
1583 0 ... 0.22 7.88 
1584 0 ... 0.22 7.88 
1585 0 ... 0.22 7.88 

 hh_hourly_weather_summary hh_precip_intensity hh_precip_probability \
1581 Possible Drizzle 0.0044 0.47 
1582 Possible Drizzle 0.0044 0.47 
1583 Clear 0.0000 0.00 
1584 Clear 0.0000 0.00 
1585 Clear 0.0000 0.00 

 hh_icon pred_demand actual_demand under_predicted_demand \
1581 rain 5.464871 6.0 0.535129 
1582 rain 5.464871 6.0 0.535129 
1583 clear-night 10.050102 7.0 -3.050102 
1584 clear-night 18.631689 14.0 -4.631689 
1585 clear-night 18.631689 14.0 -4.631689 

 pred_prob 
1581 0.207313 
1582 0.207313 
1583 0.011030 
1584 0.014190 
1585 0.014190 

[5 rows x 30 columns]
in y_train, pct of ones 0.23195392675377477 pct of zeros 0.7680460732462252
in y_test, pct of ones 0.17666254556828864 pct of zeros 0.8233374544317114
Test Log Loss: 0.36791642270654373
Train Log Loss: 0.3860038261985371
Test Accuracy Score: 0.8429191738791643
Train Accuracy Score: 0.8249996941241768
Test ROC AUC: 0.8097484420349478
Train ROC AUC: 0.8498189543625759
Test PR AUC: 0.5155636143779397
Train PR AUC: 0.6309484642903571
Test MAE: 0.15708082612083576
Train MAE: 0.1750003058758231

In [54]:
with open('/dbfs/chizhang/variance_reduction/model/071520_flf_prediction_variance_reduction_ppl.dill', 'wb') as f:
  dill.dump(ppl_vanilla, f)

### Testing

In [56]:
residual_std_m1, y_std = eval_model(ppl_best, X_test, y_test)

### Important features

In [58]:
dictionary = dict(zip(features, ppl_best['clf'].feature_importances_))

dictionary = {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1])}

# dictionary = sorted(dictionary.items(), key=operator.itemgetter(1))
print(dictionary)

In [59]:
with open('/dbfs/chizhang/variance_reduction/model/061520_flf_prediction_variance_reduction_ppl.dill', 'wb') as f:
  dill.dump(ppl_best, f)

In [60]:
##### END ####








## validate correlation in unit level

In [62]:
with open('/dbfs/chizhang/variance_reduction/model/051620_flf_prediction_variance_reduction_ppl.dill', 'rb') as f:
  ppl_best = dill.load(f)

In [63]:
for f in ['store_starting_point_id', 'submarket_id', 'window_id']:
  print('feature', f)
  print('\tTrain PearsonrResult', pearsonr(ppl_best.named_steps["target_encode"].transform(X_test)[f].values, y_test.values.reshape(len(y_test), )))

In [65]:
df.groupby(['store_starting_point_id', 'daypart']).agg({'is_flf_above_ideal' : 'mean'})

## Model training

In [67]:
# # features_high_corr = ['STORE_STARTING_POINT_ID', 'SUBMARKET_ID', 'hour_of_day', 'day_of_week'] #better residual std without day_of_week

# features_high_corr = features
# #model 1: default with balancing
# clf_default_is_unbalanced = LGBMClassifier(is_unbalance =True)#is_unbalance =True
# clf_default_is_unbalanced.fit(X_train_encoded[features_high_corr], y_train)

# #model 2: default
# clf_default = LGBMClassifier()
# clf_default.fit(X_train_encoded[features_high_corr], y_train)

In [68]:
# # model 3: grid search
# gridParams = {
#     'learning_rate': [0.01, 0.05, 0.1],
#     'num_leaves': [16, 31, 64, 128],
#     'boosting_type' : ['gbdt', 'dart', 'goss'],
#     'objective' : ['binary'],
#     'max_depth' : [-1, 5, 10],
# #     'random_state' : [501], 
# #     'colsample_bytree' : [0.5,0.7],
# #     'subsample' : [0.5,0.7],
# #     'min_split_gain' : [0.01],
#     'min_data_in_leaf':[5, 10, 20],
# #     'metric':['auc']
#     }
# #{'boosting_type': 'goss', 'objective': 'binary', 'learning_rate': 0.1, 'num_leaves': 64, 'max_depth': -1}

# clf_tuned = LGBMClassifier()
# grid = GridSearchCV(clf_tuned, gridParams,verbose=2, n_jobs=-1)
# grid.fit(X_train_encoded[features_high_corr], y_train)

# # print(grid.best_score_)
# print(grid.best_params_)


In [69]:
# # model 4: random search
# import scipy.stats as stats
# randomParams = {
#     'learning_rate': stats.uniform(0.01, 0.1), 
#     'num_leaves': stats.randint(16, 256),
#     'boosting_type' : ['gbdt', 'dart', 'goss'],
#     'objective' : ['binary'],
#     'max_depth' : [-1, 5, 10],
# #     'random_state' : [501], 
# #     'colsample_bytree' : [0.5,0.7],
# #     'subsample' : [0.5,0.7],
# #     'min_split_gain' : [0.01],
# #     'min_data_in_leaf':[10],
# #     'metric':['auc']
#     }
# #{'boosting_type': 'goss', 'objective': 'binary', 'learning_rate': 0.1, 'num_leaves': 64, 'max_depth': -1}

# clf_tuned_random = LGBMClassifier()
# model_random_search = RandomizedSearchCV(clf_tuned_random, randomParams, verbose=2, n_jobs=-1, n_iter=100)
# model_random_search.fit(X_train_encoded[features_high_corr], y_train)

# # print(grid.best_score_)
# print(model_random_search.best_params_)


## Testing

In [71]:
residual_std_m1, _ = eval_model(clf_default_is_unbalanced, X_test_encoded[features_high_corr], y_test)

In [72]:
residual_std_m2, _ = eval_model(clf_default, X_test_encoded[features_high_corr], y_test)

In [73]:
residual_std_m3, _ = eval_model(grid, X_test_encoded[features_high_corr], y_test)

In [74]:
residual_std_m4, y_std = eval_model(model_random_search, X_test_encoded[features_high_corr], y_test)

## Result plotting

In [76]:
from matplotlib import pyplot as plt
import matplotlib.ticker as mtick

# plot 4 model performance
# std_clf_is_unbalanced = 0.4239141851690449
# std_clf_default = 0.422724398281608
# std_grid_search = 0.4211761747709921
# std_random_search = 0.42077421076528226
# std_random_search_new = 0.4200526230592615 #new features
# std_testing_data = 0.4555727985659514

std_testing_data = y_std

std_clf_is_unbalanced = residual_std_m1
std_clf_default = residual_std_m2
std_grid_search = residual_std_m3
std_random_search = residual_std_m4

heights = [std_testing_data, std_clf_is_unbalanced, std_clf_default, std_grid_search, std_random_search]
heights =[i for i in heights]
heights_pct = [1 - i/std_testing_data for i in heights]
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(range(len(heights)), heights)
ax.set_ylabel('STD')

plt.xticks(range(len(heights)), ['y std', 'residual_m1 (w/ balancing)', 'residual_m2(w/o balancing)', 'residual_m3(w/ grid search)', 'residual_m4(w/ random search)'])
plt.xticks(rotation=10)
ax2 = ax.twinx()
ax2.plot(heights_pct, '-o', color='black')
ax2.set_ylabel('% of STD Reduction')

vals = ax2.get_yticks()
ax2.set_yticklabels(['{:,.2%}'.format(x) for x in vals])

for i,j in zip(range(len(heights_pct)), heights_pct):
    ax2.annotate('{:,.2%}'.format(j),xy=(i,j))
plt.grid()
display(fig)